## MaskGCT: Zero-Shot Text-to-Speech with Masked Generative Codec Transformer

[![arXiv](https://img.shields.io/badge/arXiv-Paper-COLOR.svg)](https://arxiv.org/abs/2409.00750) [![hf](https://img.shields.io/badge/%F0%9F%A4%97%20HuggingFace-model-yellow)](https://huggingface.co/amphion/maskgct) [![hf](https://img.shields.io/badge/%F0%9F%A4%97%20HuggingFace-demo-pink)](https://huggingface.co/spaces/amphion/maskgct) [![readme](https://img.shields.io/badge/README-Key%20Features-blue)](https://github.com/open-mmlab/Amphion/tree/main/models/tts/maskgct)

## Quickstart

**Clone and install**




In [ ]:
!git clone https://github.com/open-mmlab/Amphion.git
!cd Amphion/
# bash ./models/tts/maskgct/env.sh
!pip install -r requirements.txt -U
!sudo apt-get update && sudo apt-get install espeak-ng



**Model download**

We provide the following pretrained checkpoints:


| Model Name          | Description   |    
|-------------------|-------------|
| [Semantic Codec](https://huggingface.co/amphion/MaskGCT/tree/main/semantic_codec)      | Converting speech to semantic tokens. |
| [Acoustic Codec](https://huggingface.co/amphion/MaskGCT/tree/main/acoustic_codec)      | Converting speech to acoustic tokens and reconstructing waveform from acoustic tokens. |
| [MaskGCT-T2S](https://huggingface.co/amphion/MaskGCT/tree/main/t2s_model)         | Predicting semantic tokens with text and prompt semantic tokens.             |
| [MaskGCT-S2A](https://huggingface.co/amphion/MaskGCT/tree/main/s2a_model)         | Predicts acoustic tokens conditioned on semantic tokens.              |

You can download all pretrained checkpoints from [HuggingFace](https://huggingface.co/amphion/MaskGCT/tree/main) or use huggingface api.




In [ ]:
# from huggingface_hub import hf_hub_download

# # download semantic codec ckpt
# semantic_code_ckpt = hf_hub_download("amphion/MaskGCT", filename="semantic_codec/model.safetensors")

# # download acoustic codec ckpt
# codec_encoder_ckpt = hf_hub_download("amphion/MaskGCT", filename="acoustic_codec/model.safetensors")
# codec_decoder_ckpt = hf_hub_download("amphion/MaskGCT", filename="acoustic_codec/model_1.safetensors")

# # download t2s model ckpt
# t2s_model_ckpt = hf_hub_download("amphion/MaskGCT", filename="t2s_model/model.safetensors")

# # download s2a model ckpt
# s2a_1layer_ckpt = hf_hub_download("amphion/MaskGCT", filename="s2a_model/s2a_model_1layer/model.safetensors")
# s2a_full_ckpt = hf_hub_download("amphion/MaskGCT", filename="s2a_model/s2a_model_full/model.safetensors")


from modelscope import snapshot_download

snapshot_download(
    'amphion/MaskGCT', allow_patterns=[
        'semantic_codec/model.safetensors',
        'acoustic_codec/model.safetensors',
        'acoustic_codec/model_1.safetensors',
        't2s_model/model.safetensors',
        's2a_model/s2a_model_1layer/model.safetensors',
        's2a_model/s2a_model_full/model.safetensors'
    ], local_dir='./ckpts'
)

snapshot_download(
    'AI-ModelScope/w2v-bert-2.0', local_dir='./ckpts/w2v-bert-2.0'
)

# 修改 models/tts/maskgct/maskgct_utils.py
# "facebook/w2v-bert-2.0"  -> "./ckpts/w2v-bert-2.0"



**Basic Usage**

You can use the following code to generate speech from text and a prompt speech.



In [ ]:
from models.tts.maskgct.maskgct_utils import *
from huggingface_hub import hf_hub_download
import safetensors
import soundfile as sf


# build model
device = torch.device("cuda:0")
cfg_path = "./models/tts/maskgct/config/maskgct.json"
cfg = load_config(cfg_path)
# 1. build semantic model (w2v-bert-2.0)
semantic_model, semantic_mean, semantic_std = build_semantic_model(device)
# 2. build semantic codec
semantic_codec = build_semantic_codec(cfg.model.semantic_codec, device)
# 3. build acoustic codec
codec_encoder, codec_decoder = build_acoustic_codec(cfg.model.acoustic_codec, device)
# 4. build t2s model
t2s_model = build_t2s_model(cfg.model.t2s_model, device)
# 5. build s2a model
s2a_model_1layer = build_s2a_model(cfg.model.s2a_model.s2a_1layer, device)
s2a_model_full =  build_s2a_model(cfg.model.s2a_model.s2a_full, device)

In [ ]:
semantic_code_ckpt = './ckpts/semantic_codec/model.safetensors'
codec_encoder_ckpt = './ckpts/acoustic_codec/model.safetensors'
codec_decoder_ckpt = './ckpts/acoustic_codec/model_1.safetensors'
t2s_model_ckpt = './ckpts/t2s_model/model.safetensors'
s2a_1layer_ckpt = './ckpts/s2a_model/s2a_model_1layer/model.safetensors'
s2a_full_ckpt = './ckpts/s2a_model/s2a_model_full/model.safetensors'


# load semantic codec
safetensors.torch.load_model(semantic_codec, semantic_code_ckpt)
# load acoustic codec
safetensors.torch.load_model(codec_encoder, codec_encoder_ckpt)
safetensors.torch.load_model(codec_decoder, codec_decoder_ckpt)
# load t2s model
safetensors.torch.load_model(t2s_model, t2s_model_ckpt)
# load s2a model
safetensors.torch.load_model(s2a_model_1layer, s2a_1layer_ckpt)
safetensors.torch.load_model(s2a_model_full, s2a_full_ckpt)

In [ ]:
# inference
save_path = "test.mp3"
prompt_wav_path = "./models/tts/maskgct/wav/prompt.wav"
prompt_text = " We do not break. We never give in. We never back down."
target_text = "In this paper, we introduce MaskGCT, a fully non-autoregressive TTS model that eliminates the need for explicit alignment information between text and speech supervision."
target_text = '这种模式本质就是吃大锅饭，地方对于干工作是没什么积极性的，干好了给国家，干不好中央给兜底，那还努力啥？'
# target_text = "你好，我是小Kitty!"
# Specify the target duration (in seconds). If target_len = None, we use a simple rule to predict the target duration.
target_len = None

maskgct_inference_pipeline = MaskGCT_Inference_Pipeline(
    semantic_model,
    semantic_codec,
    codec_encoder,
    codec_decoder,
    t2s_model,
    s2a_model_1layer,
    s2a_model_full,
    semantic_mean,
    semantic_std,
    device,
)

recovered_audio = maskgct_inference_pipeline.maskgct_inference(
    prompt_wav_path, prompt_text, target_text, "en", "zh", target_len=target_len
)
sf.write(save_path, recovered_audio, 24000)

In [ ]:
from IPython.display import display, Audio

display(Audio(save_path, autoplay=True))

In [ ]:
from IPython.display import display, Audio

display(Audio('./models/tts/maskgct/wav/prompt.wav', autoplay=True))